In [70]:
import os
import nibabel as nib
import numpy as np
from scipy.ndimage.interpolation import zoom
import scipy as sp
from tqdm import tqdm
from tqdm.notebook import tqdm_notebook

import keras
from keras.models import Model
from keras.layers import Input, Dense, Reshape, Flatten, LeakyReLU, Dropout, Embedding, Concatenate
from keras.layers.core import Activation
from keras.layers.convolutional import Conv3D, Deconv3D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.utils.vis_utils import plot_model


def load_nifti(file_path, mask=None, z_factor=None, remove_nan=False):
    """Load a 3D array from a NIFTI file."""
    img = nib.load(file_path)
    struct_arr = np.array(img.get_data())

    if remove_nan:
        struct_arr = np.nan_to_num(struct_arr)
    if mask is not None:
        struct_arr *= mask
    if z_factor is not None:
        struct_arr = np.around(zoom(struct_arr, z_factor), 0)

    return struct_arr


def save_nifti(file_path, struct_arr):
    """Save a 3D array to a NIFTI file."""
    img = nib.Nifti1Image(struct_arr, np.eye(4))
    nib.save(img, file_path)
    




In [2]:
def prepareData(use_smooth = False, runningOnServer = True):
    rootDir = 'C:/Users/Eshan/Google Drive UALBERTA/Data/' if not runningOnServer else '/mnt/hdd1/lxc-hdd1/tahjid/PD Data/'
    patientList, patientNumbers, dataset = [], [], []
    labelMap = dict(Control=0, PD=1)
    typeMap = dict(FullScan=0, GrayMatter=1, WhiteMatter=2)
    fullScanPath = rootDir + 'FinalData/'
    wmgmpath = rootDir + 'FinalDataWMGM/' if not use_smooth else rootDir + 'FinalDataWMGMSmooth/'
    prefix = 'mwp' if not use_smooth else 'smwp'
    ext = '.nii'
    for i in tqdm(['Control', 'PD']):
        path = fullScanPath + i + '/'
        listOfFiles = [f for f in os.listdir(path) if f.endswith(ext)]
        for file in tqdm(listOfFiles):
            patientNumbers.append(file[:4])
        path = wmgmpath + i + '/'
        listOfFiles = [f for f in os.listdir(path) if f.endswith(ext)]
        for file in tqdm(listOfFiles):
            filename = file[4:8] if not use_smooth else file[5:9]
            if filename not in patientNumbers:
                continue
            if not use_smooth:
                patientList.append([i, file[4:8]])
            else:
                patientList.append([i, file[5:9]])

    for i in tqdm(patientList):
        path = fullScanPath + i[0] + '/'
        patientIdVal = i[1]
        fullScanvalue = os.path.join(path + i[1] + ext)
        path = wmgmpath + i[0] + '/'
        gmval = os.path.join(path + prefix + str(typeMap['GrayMatter']) + i[1] + ext)
        wmval = os.path.join(path + prefix + str(typeMap['WhiteMatter']) + i[1] + ext)
        labelval = labelMap[i[0]]
        dataset.append([patientIdVal,fullScanvalue, gmval, wmval, labelval])
    return np.array(dataset)
dataset = prepareData()

100%|██████████| 299/299 [00:00<00:00, 1518277.11it/s]

100%|██████████| 626/626 [00:00<00:00, 248521.94it/s]

100%|██████████| 299/299 [00:00<00:00, 2696982.57it/s]

100%|██████████| 1170/1170 [00:00<00:00, 279492.86it/s]


In [3]:
from sklearn.model_selection import train_test_split
X = dataset[:,:4]
y = dataset[:,4:]

In [4]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=.20, random_state=42)


In [10]:
dataset[:,:1]

array([['3554'],
       ['5127'],
       ['5037'],
       ...,
       ['3834'],
       ['4109'],
       ['3574']], dtype='<U68')

In [5]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data, labels, batch_size=2, dim1=(91,109,91), dim2=(121,145,121) , n_channels=1,
                 n_classes=2, shuffle=True):
        'Initialization'
        self.dim1 = dim1
        self.dim2 = dim2
        self.batch_size = batch_size
        self.labels = list(labels)
        self.data = data
        self.n_channels = n_channels
        self.n_classes = n_classes
        self.shuffle = shuffle
        self.list_IDs = list(data[:,:1])
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # print(indexes)
        # Find list of IDs
        # list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        fs, gm, wm, y = self.__data_generation(indexes)

        return fs, gm, wm, y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle:
            np.random.shuffle(self.indexes)

    def __data_generation(self, indexes):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        fs = np.empty((self.batch_size, *self.dim1, self.n_channels))
        gm = np.empty((self.batch_size, *self.dim2, self.n_channels))
        wm = np.empty((self.batch_size, *self.dim2, self.n_channels))
        y = np.empty((self.batch_size), dtype=int)

        # Generate data
        for i, count in enumerate(indexes):
            val = self.data[count]
            fullscan = load_nifti(val[1])
            graymatter = load_nifti(val[2])
            whitematter = load_nifti(val[3])
            # print(fullscan.shape, graymatter.shape, whitematter.shape)
            # break
            # Store sample
            fs[i,] = fullscan[..., np.newaxis]
            gm[i,] = graymatter[..., np.newaxis]
            wm[i,] = whitematter[..., np.newaxis]


            # Store class
            y[i] = self.labels[i]

        return fs, gm, wm, keras.utils.to_categorical(y, num_classes=self.n_classes)
    

In [6]:
d = DataGenerator(x_train, y_train)


In [23]:
a = d.__getitem__(0)
b = np.concatenate((a[1][0].reshape(121,145,121), a[1][0].reshape(121,145,121)), axis=0)

/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which 

In [28]:
def plot(img, savefig = False, path = "", name = ""):
    from nilearn import plotting
    display = plotting.plot_anat(img)
    if savefig: display.savefig(path + "/" + name)
    plotting.show()
    display.close()     

In [39]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

v = a[1][0].reshape(121,145,121)



In [47]:
b.shape

(242, 145, 121)

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111,projection = '3d')
ax.plot(v[:,0], v[:,1], v[:,2])
plt.show()

In [78]:
def generator(phase_train=True, n_classes = 2 ,params={'z_size':500, 'strides':(11,5,11), 'kernel_size':(22,29,11)}):
    """
    Returns a Generator Model with input params and phase_train
    Args:
        phase_train (boolean): training phase or not
        params (dict): Dictionary with model parameters
    Returns:
        model (keras.Model): Keras Generator model
    """

    z_size = params['z_size']
    strides = params['strides']
    kernel_size = params['kernel_size']
    
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(z_size)(li)
    li = Reshape((1,1,1,z_size))(li)
    
    
    inputs = Input(shape=(1, 1, 1, z_size))
    
    
    merge_inputs = Concatenate()([inputs, li])
    
    g1 = Deconv3D(filters=1024, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='valid')(merge_inputs)
    g1 = BatchNormalization()(g1, training=phase_train)
    g1 = Activation(activation='relu')(g1)

    g2 = Deconv3D(filters=512, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g1)
    g2 = BatchNormalization()(g2, training=phase_train)
    g2 = Activation(activation='relu')(g2)

    g3 = Deconv3D(filters=264, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g2)
    g3 = BatchNormalization()(g3, training=phase_train)
    g3 = Activation(activation='relu')(g3)

    g4 = Deconv3D(filters=128, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g3)
    g4 = BatchNormalization()(g4, training=phase_train)
    g4 = Activation(activation='relu')(g4)

    g5 = Deconv3D(filters=1, kernel_size=kernel_size,
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(g4)
    g5 = BatchNormalization()(g5, training=phase_train)
    g5 = Activation(activation='sigmoid')(g5)

    model = Model(inputs=[inputs, in_label], outputs=g5)
    model.summary()

    return model

In [80]:
model = generator()
# model.summary()
# plt.show()
plot_model(model, to_file = "generator_plot.png", show_shapes = True, show_layer_names = True)
# 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 1)            0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, 1, 50)        100         input_21[0][0]                   
__________________________________________________________________________________________________
dense_5 (Dense)                 (None, 1, 500)       25500       embedding_5[0][0]                
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 1, 1, 1, 500) 0                                            
__________________________________________________________________________________________________
reshape_4 

In [98]:
def discriminator(phase_train = True, n_classes = 2, params={'cube_len':64, 'strides':(11,5,11), 'kernel_size':(22,29,11), 'leak_value':0.2}):
    """
    Returns a Discriminator Model with input params and phase_train 
    Args:
        phase_train (boolean): training phase or not
        params (dict): Dictionary with model parameters    
    Returns:
        model (keras.Model): Keras Discriminator model
    """
    cube_len = params['cube_len']
    strides = params['strides']
    kernel_size = params['kernel_size'] 
    leak_value = params['leak_value']
    
    in_label = Input(shape=(1,))
    li = Embedding(n_classes, 50)(in_label)
    n_nodes = 242 * 145 * 141
    li = Dense(n_nodes)(li)
    li = Reshape((242, 145, 141, 1))(li)
    
    inputs = Input(shape=(242, 145, 141, 1))
    
    merge_inputs = Concatenate()([inputs, li])
    
    d1 = Conv3D(filters=128, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(merge_inputs)
    d1 = BatchNormalization()(d1, training=phase_train)
    d1 = LeakyReLU(leak_value)(d1)

    d2 = Conv3D(filters=256, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(d1)
    d2 = BatchNormalization()(d2, training=phase_train)
    d2 = LeakyReLU(leak_value)(d2)

    d3 = Conv3D(filters=512, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(d2)
    d3 = BatchNormalization()(d3, training=phase_train)
    d3 = LeakyReLU(leak_value)(d3)

    d4 = Conv3D(filters=1024, kernel_size=kernel_size,
                  strides=strides, kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='same')(d3)
    d4 = BatchNormalization()(d4, training=phase_train)
    d4 = LeakyReLU(leak_value)(d4)

    d5 = Conv3D(filters=1, kernel_size=(1,1,1),
                  strides=(1, 1, 1), kernel_initializer='glorot_normal',
                  bias_initializer='zeros', padding='valid')(d4)
    d5 = BatchNormalization()(d5, training=phase_train)
    d5 = Activation(activation='sigmoid')(d5) 
    
    d6 = Flatten()(d5)
    d6 = Dropout(0.4)(d6)
    d6 = Dense(1, activation='sigmoid')(d6)
    model = Model(inputs=[inputs, in_label], outputs=d6)
    model.summary()

    return model

In [99]:
dis = discriminator()
plot_model(dis, to_file = "discriminator_plot.png", show_shapes = True, show_layer_names = True)

/mnt/hdd1/lxc-hdd1/tahjid/anaconda3/envs/tf-gpu/lib/python3.6/site-packages/keras/engine/network.py:180: UserWarning: Model inputs must come from `keras.layers.Input` (thus holding past layer metadata), they cannot be the output of a previous non-Input layer. Here, a tensor specified as input to your model was not an Input tensor, it was generated by layer concatenate_9.
Note that input tensors are instantiated via `tensor = keras.layers.Input(shape)`.
The tensor that caused the issue was: concatenate_9/concat:0
  str(x.name))


ValueError: Graph disconnected: cannot obtain value for tensor Tensor("input_36:0", shape=(?, 1), dtype=float32) at layer "input_36". The following previous layers were accessed without issue: []